# Internet Computer

## Deploy

Deploy `basic_solana` example on mainnet.

In [ ]:
!dfx identity use admin

In [ ]:
!dfx deploy --ic basic_solana

In [ ]:
!dfx canister status --ic --all


## Create a new identity
Create a new identity, which is not a controller of the `basic_solana` canister, to interact with it.

In [30]:
!dfx identity new helium_demo

Your seed phrase for identity 'helium_demo': snake apology river letter warrior pool bonus boring sing shed slide arch fix excuse knife abuse monster useless manage anxiety shine urban feel manual
This can be used to reconstruct your key in case of emergency, so write it down in a safe place.
Created identity: "helium_demo".


In [31]:
!dfx identity use helium_demo

Using identity: "helium_demo".


In [32]:
!dfx identity get-principal

hopjo-r5uca-qhtpi-owytc-7loj3-h2xel-wxdqf-htpwz-7gxq7-rqdlc-vae


In [41]:
%env MY_PRINCIPAL="hopjo-r5uca-qhtpi-owytc-7loj3-h2xel-wxdqf-htpwz-7gxq7-rqdlc-vae"

env: MY_PRINCIPAL="hopjo-r5uca-qhtpi-owytc-7loj3-h2xel-wxdqf-htpwz-7gxq7-rqdlc-vae"


## Read Balance

In [33]:
%env SOLANA_CLI_ADDRESS="DTXBMjo6tcosChHeV83Aq6gZuH8K7JYA6EL4ADaMi3Zc"

env: SOLANA_CLI_ADDRESS="DTXBMjo6tcosChHeV83Aq6gZuH8K7JYA6EL4ADaMi3Zc"


In [34]:
!dfx canister call basic_solana get_balance "opt (${SOLANA_CLI_ADDRESS})" --ic


(8_770_883_522 : nat)


### Explanations

* SOL has 9 decimals: 1 SOL == `1_000_000_000` Lamports
* [Logs of SOL RPC canister](https://titvo-eiaaa-aaaar-qaogq-cai.raw.icp0.io/logs?sort=desc)
    * 3 [HTTPS outcalls](https://support.dfinity.org/hc/en-us/articles/12342341722260-What-is-an-HTTPS-Outcall):
        * `solana-devnet.g.alchemy.com`
        * `lb.drpc.org`
        * `devnet.helius-rpc.com`
    * each HTTPS outcall involves each replica in the subnet making an HTTPS request => 34 HTTPs request for the [fiduciary subnet](https://dashboard.internetcomputer.org/subnet/pzp6e-ekpqk-3c5x7-2h6so-njoeq-mt45d-h3h6c-q3mxf-vpeq5-fk5o7-yae)
    *  TOTAL: 102 HTTPS requests distributed all-over the world! 💪
    * **no single point of failure** 🛡️



*Request airdrop with Solana CLI*


In [28]:
!dfx canister call basic_solana get_balance "opt (${SOLANA_CLI_ADDRESS})" --ic


(8_770_883_522 : nat)


## Send SOL

* Canister can control an Ed25519 public/private key pair thanks to [threshold Schnorr](https://internetcomputer.org/docs/building-apps/network-features/signatures/t-schnorr/).
    * Each replica holds a share of a secret key
    * Each replica can use its share to create a signature share for a given message
    * Combining enough signature shares result in a standard signature for that message
    * ℹ️ At no moment is the secret key reconstructed.

### Fund address

In [35]:
!dfx canister call basic_solana solana_account --ic

("G7Tgmmi5y4KMByBULWnu6JHaukszKVQRgqCgVy8bKbPX")


In [36]:
%env SOLANA_CANISTER_ADDRESS="G7Tgmmi5y4KMByBULWnu6JHaukszKVQRgqCgVy8bKbPX"

env: SOLANA_CANISTER_ADDRESS="G7Tgmmi5y4KMByBULWnu6JHaukszKVQRgqCgVy8bKbPX"


In [37]:
!dfx canister call basic_solana get_balance --ic

(100_000_000 : nat)


*Fund address via Solana CLI*

In [38]:
!dfx canister call basic_solana get_balance --ic


(100_000_000 : nat)


### Send transaction

Send transaction of 1 Lamport back to Solana CLI address

In [43]:
!dfx canister call basic_solana send_sol "(opt principal $MY_PRINCIPAL, $SOLANA_CLI_ADDRESS, 1 : nat)" --ic

(
  "5EiYrSeDuc4TcQe67PoGdTaMf2tGqZt1rusmF4kpegpuoer78qfNhLyf25dSkZMiDDqPHCBAtkiCrfhmGuLsV9uz",
)


Check transaction on [Solana explorer](https://explorer.solana.com/?cluster=devnet):
* nothing distinguishes a transaction signed via threshold Ed25529 from a regular transaction
* address controlled by the canister also paid the transaction fee

In [44]:
!dfx canister call basic_solana get_balance --ic


(99_994_999 : nat)


* 1 major difficulty is getting a recent blockhash that must be part of the transaction.
* See the [code](https://github.com/dfinity/sol-rpc-canister/blob/8e7e32f50a2e05fa671a3df8f7b19f64c5abbdce/examples/basic_solana/src/main.rs#L226) for more details.

## Send USDC

Send 1 USDC from addressed controlled by canister to my Solana CLI wallet

### Fund address controlled by canister

* USDC devnet: [4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU](https://explorer.solana.com/address/4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU?cluster=devnet)
* USDC [faucet](https://faucet.circle.com/)

In [45]:
%env USDC_DEVNET="4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU"

env: USDC_DEVNET="4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU"


In [47]:
!dfx canister call basic_solana associated_token_account "(opt principal $MY_PRINCIPAL, $USDC_DEVNET)" --ic


("4TwxHLegFAHR8PPQTVqGWZtsQsoCjqcYinDKjaszPBmF")


In [51]:
!dfx canister call basic_solana get_spl_token_balance "(null, $USDC_DEVNET)" --ic


(
  record {
    decimals = 6 : nat8;
    uiAmount = opt (10.0 : float64);
    uiAmountString = "10";
    amount = "10000000";
  },
)


In [52]:
!dfx canister call basic_solana send_spl_token "(opt principal $MY_PRINCIPAL, $USDC_DEVNET, $SOLANA_CLI_ADDRESS, 1_000_000 : nat)" --ic

(
  "5RCxoPgR1XeUctX2a7GyW64TVpQNggnCtRKRrPfLpB7A2QYSAP25EWq1YKUcFKQx3pyxMpk98MzG4VdMUPPUNWFH",
)
